In [0]:


import pandas as pd
from pyspark.sql import functions as F
from datetime import datetime
from applications.db_check_v1.common.db_check_utils import query
from conf.settings import PG_USAGE_HOSTS, PG_USAGE_NAME, PG_USAGE_ACCESS_ID, PG_USAGE_SECRET_KEY, \
    CITUS_USAGE_NAME, CITUS_USAGE_ACCESS_ID, CITUS_USAGE_HOSTS, CITUS_USAGE_SECRET_KEY

PLPROXY_DSN = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_USAGE_NAME,
        user=PG_USAGE_ACCESS_ID,
        host=PG_USAGE_HOSTS[0][0],
        password=PG_USAGE_SECRET_KEY,
        port=PG_USAGE_HOSTS[0][1]
    )
)

def get_date_list(begin_date, end_date, freq):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list


begin_date = datetime(2019, 1, 1)
end_date = datetime(2020, 8, 31)

DATE_GRANULARITY_MAPPINGLIST = {
    "daily": get_date_list(begin_date, end_date, "D"),
    "weekly": get_date_list(begin_date, end_date, "W-SAT"),
    "monthly": get_date_list(begin_date, end_date, "M")
}


DATE_GRANULARITY_MAPPINGLIST["monthly"].reverse()

In [0]:

basic_dump="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-mu_app.v2/fact/granularity=monthly/"
basic="s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v1/fact/granularity=monthly/date=2017-01-31"
segment_by_age_gender_sql="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-ag_app.v2/fact/granularity=/"
segment_by_product_sql="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-au_app.v2/fact/granularity=weekly/"
app_cross_app="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-ca_app.v2/fact/granularity=monthly/"
app_retention="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-rt_app.v2/fact/granularity=monthly/"
start_str="2015-01"
end_str="2015-05"
basic_dump=spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-mu_app.v2/fact").parquet(basic_dump).where("month>='{}'".format(start_str)).where("month<='{}'".format(end_str))
# basic_dump=spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-mu_app.v2/fact").parquet(basic_dump).where("month>='{}'".format(start_str)).where("month<='{}'".format(end_str))
# df_ag = spark.read.format('delta').load(basic).where("date>='{}'".format(start_str)).where("date<='{}'".format(end_str))
# segment_by_product_sql = spark.read.format('delta').load(segment_by_product_sql)
# print("count:"+str(basic_dump.count().show()))
basic_dump.createOrReplaceTempView("basic_dump")
# print("segment_by_product_sql.count(): "+str(segment_by_product_sql.filter("kpi=1").groupBy("date").count().orderBy("date").show(1500,truncate=False)))

basic_dump= spark.sql("""
                SELECT date,count(1) AS count
                FROM basic_dump
                WHERE kpi=1
                and estimate is not null
                and estimate <> 0
                GROUP BY date
                ORDER BY date
                """)
basic_dump.show(1000,False)